# Parte I

In [261]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from prettytable import PrettyTable, NONE

In [262]:
np.random.seed(3)

In [263]:
dataset = np.fromfile("./data/dataAll.txt", dtype=np.float64)

In [264]:
print(dataset.shape)
print(dataset)

(3000,)
[-363.7884  244.1423    0.     ... -140.147   734.0983    0.    ]


O dataset encontra-se atualmente em um array unidimensional com 3000 elementos, organizados sequencialmente em grupos de 3. Isso indica a necessidade de reestruturar o array para uma forma matricial com 3 colunas, de modo a viabilizar uma manipulação mais eficiente e semântica dos dados.

## Redimensionando o dataset

A função reshape é utilizada para modificar a estrutura de um array sem alterar seus dados. Na forma em que está sendo aplicada, o valor 3 especifica o novo número de colunas, enquanto o parâmetro -1 instrui o NumPy a calcular automaticamente a quantidade de linhas necessária com base no total de elementos disponíveis.

In [265]:
dataset = dataset.reshape(-1,3)

In [266]:
dataset.shape
print(dataset)

[[-363.7884  244.1423    0.    ]
 [ 328.7572 -256.7658    1.    ]
 [-164.9827  613.2164    0.    ]
 ...
 [ 872.4311  669.1192    1.    ]
 [ 822.6894 -702.6489    1.    ]
 [-140.147   734.0983    0.    ]]


# Adicionando o viés ($x_0 = -1$)

Criando um array coluna que possui os valores de -1 no neuronio perceptron 

In [267]:
vies = -1 * np.ones((dataset.shape[0], 1))
print(vies.shape)
print(type(vies))

(1000, 1)
<class 'numpy.ndarray'>


O array coluna é incorporado ao dataset por meio da função np.hstack, a qual realiza a concatenação horizontal de arrays, unindo a nova coluna à matriz existente ao longo do eixo das colunas.

In [268]:
valores = np.hstack((vies, dataset))

print(valores)

[[  -1.     -363.7884  244.1423    0.    ]
 [  -1.      328.7572 -256.7658    1.    ]
 [  -1.     -164.9827  613.2164    0.    ]
 ...
 [  -1.      872.4311  669.1192    1.    ]
 [  -1.      822.6894 -702.6489    1.    ]
 [  -1.     -140.147   734.0983    0.    ]]


Com o ajuste dos dados, preocupa-se agora com a criação de uma classe perceptron capaz de tratar esses dados 

## Modelando o Perceptron

O Perceptron é um modelo de rede neural simples que é usado para classificação binária. Foi proposto por Frank Rosenblatt em 1958 e é um dos primeiros algoritmos de redes neurais. Neste notebook representaremos ele através dessa classe.

In [269]:
class Perceptron:
    def __init__(self, intervalo, ativacao, tamanho_input):
        self.intervalo = intervalo
        self.ativacao = ativacao
        self.pesos = np.random.uniform(self.intervalo[0], self.intervalo[1], size=tamanho_input)
        self.ajustes = 0

    def __str__(self):
        cabecalho = "Perceptron de Rosenblatt\n"
        numero_ajustes = f"Número de ajustes: {self.ajustes}\n"
        pesos = f"Pesos: w0 = {self.pesos[0]}; w1 = {self.pesos[1]}; w2 = {self.pesos[2]}\n"
        return (cabecalho + numero_ajustes + pesos)

    def get_pesos(self):
        return self.pesos
    
    def get_ajustes(self):
        return self.ajustes
    
    def ajuste_pesos(self, novos_pesos):
        self.pesos = novos_pesos
        self.ajustes += 1
    
    def foward(self, X):
        u = np.dot(X, self.pesos)
        y = self.ativacao(u)
        return y



## Regra Delta

A Regra Delta é um algoritmo de aprendizado supervisionado usado para ajustar os pesos de um neurônio artificial (como o Perceptron) durante o treinamento.

Ela se baseia na ideia de minimizar o erro entre a saída desejada e a saída do neurônio, ajustando os pesos com base nesse erro.

In [270]:
def regra_delta(omega, x_n, erro, eta=0.1):
    novos_pesos = omega + eta * erro * x_n
    return novos_pesos

## Função de treino

In [271]:
def train(valores, neuronio: Perceptron, eta=0.1):
    epocas = 0
    aprender = True

    while aprender:
        epocas += 1
        aprender = False
        for val in valores:
            x_n = val[:3]
            y_chapeu = neuronio.foward(x_n)
            y_d = val[3]
            e = y_d - y_chapeu
            if int(e) != 0:
                aprender = True
                novos_pesos = regra_delta(neuronio.get_pesos(), x_n, e, eta)
                neuronio.ajuste_pesos(novos_pesos)
    
    return (neuronio.get_ajustes(), epocas)


## Função de Ativação

A função de ativação solicitada é a função degrau, a qual é representada pela seguinte expressão:

In [272]:
def degrau(x, tetha=0):
    if x-tetha < 0:
        return 0
    return 1

## Inicialização

Instanciando o perceptron com os valores solicitados.

In [ ]:
tamanho_input = valores.shape[1] - 1 
neuronio = Perceptron((-0.5, 0.5), degrau, tamanho_input)

In [ ]:
print(neuronio)

## Treinamento

In [ ]:
ajustes, epocas = train(valores, neuronio, eta=0.1)

In [ ]:
print(f"O treinamento terminou em {epocas} épocas e com {ajustes} ajustes nos pesos")

In [ ]:
print(neuronio)

In [ ]:
w0, w1, w2 = neuronio.get_pesos()

In [ ]:
cores = np.where(dataset[:, 2] == 0, 'red', 'blue')

plt.scatter(dataset[:, 0], dataset[:, 1], c=cores, s=1)

x = np.linspace(-1000, 1000, 100)
y = w0/w2 - x * w1/w2
plt.plot(x, y, c='k')

legenda = [
       Line2D([0], [0], marker='o', color='w', label='Classe 0',
           markerfacecolor='red', markersize=4),
       Line2D([0], [0], marker='o', color='w', label='Classe 1',
           markerfacecolor='blue', markersize=4),
       Line2D([], [], color='k', linestyle='-', label='Reta resultante')
]
plt.legend(handles=legenda)
plt.title("Resultado do treinamento do neurônio Perceptron")
plt.show()

# Parte II

In [ ]:
dataset2 = np.fromfile("./data/data0.txt", dtype=np.float64)

In [ ]:
dataset2 = dataset2.reshape(-1,3)

In [ ]:
dataset2.shape

### Adicionando o viés ($x_0 = -1$)

In [ ]:
vies = -1 * np.ones((dataset2.shape[0], 1))

In [ ]:
valores2 = np.hstack((vies, dataset2))

## Teste de diferentes confugurações de $\eta$ e $I$

In [ ]:
def grid_search(etas, intervalos):
    g1, g2 = np.meshgrid(etas, intervalos[:,0])
    _, g3 = np.meshgrid(etas, intervalos[:,1])
    grid = np.hstack([g1.reshape(-1, 1), g2.reshape(-1, 1), g3.reshape(-1, 1)]) # Resulta no produto carteziano etas X intervalos [[eta, -lim, +lim], ...]

    tabela = PrettyTable()
    tabela.vrules = NONE
    tabela.title = 'Configurações'
    tabela.field_names = [["Taxa de Aprendizado"], "Intervalo de Pesos", "Média dos Ajustes", "Desvio Padrão dos Ajustes", "Menor número de épocas para a convergência"]

    for configuracao in grid:
        eta = configuracao[0]
        intervalo = configuracao[1:3]
        print(f"Configuração - Taxa de aprendizado: {eta}, Intervalo: {intervalo}")

        ajustes = np.array([])
        menor_n_epocas = np.inf
        print("Treinos")
        for i in range(10):
            print(f"{i + 1}/10", end='\r')
            neuro = Perceptron(intervalo, degrau, valores2.shape[1] - 1)
            ajuste, epocas = train(valores2, neuro, eta)

            ajustes = np.insert(ajustes, 0, ajuste)
            
            if epocas < menor_n_epocas:
                menor_n_epocas = epocas
        print()
        
        tabela.add_row([eta, (float(intervalo[0]), float(intervalo[1])), ajustes.mean(), ajustes.std(), menor_n_epocas])

    print("\n Último neurônio:")
    print(neuro)
    return tabela


In [ ]:
etas = np.array([0.4, 0.1, 0.01])
intervalos = np.array([(-100, 100), (-0.5, 0.5)])

In [ ]:
tabela = grid_search(etas, intervalos)

### Resultados

In [ ]:
print(tabela)

### Análise dos Resultados

Para os casos em que os pesos são inicializados em um intervalo maior $(-100, +100)$, podemos notar que tanto o número de ajustes quanto o número de épocas para a convergência tendem a ser maiores. Os resultados também mostram que, quanto menor a taxa de aprendizado, maior é o número de ajustes necessários para a convergência.

No entanto, o menor número médio de ajustes para casos com pesos inicializados entre $(-0.5, +0.5)$ não é uma regra geral. No caso do experimento, isso só foi verdade porque existia uma solução dentro desse intervalo. Caso escolhêssemos um intervalo pequeno, mas distante de uma solução, o número de ajustes seria maior. No caso de não sabermos a região onde possivelmete existe uma solução, a melhor alternativa é escolher um intervalo maior. 